In [4]:
from __future__ import (absolute_import, print_function, division)
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd
import os
from scipy.stats import dirichlet
import numpy as np
from scipy.stats import wishart
from scipy.stats import norm
# import numba
from scipy.stats import multivariate_normal as mvn
from numpy.linalg import inv
from numpy import log as ln
from scipy.special import psi
from numpy.random import random as rand
%matplotlib 

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
Vendor:  Continuum Analytics, Inc.


Using matplotlib backend: MacOSX


Package: mkl
Message: trial mode expires in 30 days


Different than EM, variational inference introduces priors on model variables (local and global)

Wishart Distribution
-----
scipy.stats.wishart

scipy.stats.wishart = <scipy.stats._multivariate.wishart_gen object at 0x4522576c>[source]
A Wishart random variable.

The df keyword specifies the degrees of freedom. The scale keyword specifies the scale matrix, which must be symmetric and positive definite. In this context, the scale matrix is often interpreted in terms of a multivariate normal precision matrix (the inverse of the covariance matrix).

Parameters:	
x : array_like
Quantiles, with the last axis of x denoting the components.
df : int
Degrees of freedom, must be greater than or equal to dimension of the scale matrix
scale : array_like
Symmetric positive definite scale matrix of the distribution
random_state : None or int or np.random.RandomState instance, optional
If int or RandomState, use it for drawing the random variates. If None (or np.random), the global np.random state is used. Default is None.
Alternatively, the object may be called (as a function) to fix the degrees
of freedom and scale parameters, returning a “frozen” Wishart random
variable:
rv = wishart(df=1, scale=1)
Frozen object with the same methods but holding the given degrees of freedom and scale fixed.

Notes

The scale matrix scale must be a symmetric positive definite matrix. Singular matrices, including the symmetric positive semi-definite case, are not supported.

The Wishart distribution is often denoted

Wp(ν,Σ)
where ν is the degrees of freedom and Σ is the p×p scale matrix.

The probability density function for wishart has support over positive definite matrices S; if S∼Wp(ν,Σ), then its PDF is given by:

$f(S)=|S|ν−p−122νp2|Σ|ν2Γp(ν2)exp(−tr(Σ−1S)/2)$
If $S∼Wp(ν,Σ)$ (Wishart) then S−1∼W−1p(ν,Σ−1) (inverse Wishart).

If the scale matrix is 1-dimensional and equal to one, then the Wishart distribution W1(ν,1) collapses to the χ2(ν) distribution.

New in version 0.16.0.


Dirichlet Distribution
-------
scipy.stats.dirichlet = <scipy.stats._multivariate.dirichlet_gen object at 0x452255cc>[source]
A Dirichlet random variable.

The alpha keyword specifies the concentration parameters of the distribution.

New in version 0.15.0.

Parameters:	
x : array_like
Quantiles, with the last axis of x denoting the components.
alpha : array_like
The concentration parameters. The number of entries determines the dimensionality of the distribution.
random_state : None or int or np.random.RandomState instance, optional
If int or RandomState, use it for drawing the random variates. If None (or np.random), the global np.random state is used. Default is None.
Alternatively, the object may be called (as a function) to fix
concentration parameters, returning a “frozen” Dirichlet
random variable:
rv = dirichlet(alpha)
Frozen object with the same methods but holding the given concentration parameters fixed.
Each α entry must be positive. The distribution has only support on the simplex defined by

$∑i=1Kxi≤1
The probability density function for dirichlet is f(x)=1B(α)∏i=1Kxαi−1i
where (α)=∏Ki=1Γ(αi)Γ(∑Ki=1αi)
and α=(α1,…,αK), the concentration parameters and K is the dimension of the space where x takes values.$



From discussion on Wishart
---
* $E[\Lambda] = aB^{-1}$ 
* $E[ln|\Lambda|] = d(d-1)/4 * ln(\pi) + d ln(2) - ln|B| + \sum_{{j=1}}^{d}\Psi(\alpha/2+(1-j)/2)$

From VI for GMM given distributions
----
* $q(\pi) = Dirichlet(\alpha_1^`, ..., \alpha_K^`)$
* $q(\mu_j) = Normal(m_j^`,\sum_j^`)$
* $q(\Lambda_j) = Wishart(\alpha_j^`, B_j^`)$
* $q(c_i) = Multinomial(\phi_i)$

The indices indicate the iterative dependence I am going to implement as a part of VI for GMM.

In [5]:
wpdf = wishart.pdf
dirpdf = dirichlet.pdf

Input:
* Data $x_1, x_2, ..., x_n$ where x $\epsilon R^d$
* k is the number of clusters as in GMM
Output:
* Parameters for $q(\pi), q(\mu_j), q(\Lambda_j), q(c_i)$ 

VI Algorithm for Gaussian Mixture Model
------
1. Initialize $\alpha_1^{(0)}, ...., \alpha_k^{(0)}) , (m_j^{(0)}, \sum_j^{(0)}), (\alpha_j^{(0)}, B_j^{(0)})$ in some way.
2. At iteration t:
    * Update $q(c_i)$ for i=1, ..., n
    * For j = 1, ...., K Set $n_j^{(t)}=\sum_{j=1}^{d} \phi_i^{t}(j)$
    * For j = 1, ..., K 
        * Update $q(\pi)$ by setting $\alpha_j^{(t)} = \alpha + n_j^{(t)}$
    * For j = 1, ...., K
        * Update $q(\mu_j)$:
            
            * $\sum_j^{(t)}$=$(c^{-1}I+n_j^{(t)}\alpha_j^{(t-1)}(B_j^{(t-1)}))^{-1}$
            * $m_j^{(t)}=\sum_j^{(t)}(\alpha_j^{(t-1)}(B_j^{(t-1)})^{-1}\sum_{{i=1}}^{d}\phi^{t}(j)x_i)$
    * For j = 1, ..., K
        * Update $q(\Lambda_j)$:
            
            * $\alpha_j^{(t)} = \alpha + n_j^{(t)}$
            * $B_j^{(t)}=B+\sum_i^{n}\phi_i^{(t)}(j)[(x_i-m_j^{(t})(x_i-m_j^{(t})^{T} + \sum_j^{(t)}]$
    * And lastly, calculate variation inference objective function L


As one can see above, there are two major loops, 1 to T for iterations in order to update and 1 to K for j. 

In [113]:
def vi_gmm(x_train, k, T):
    # Initialization
    n, d = x_train.shape
    A_0 = x_train.cov() # empirical covariance
    alpha_k = np.ones((T, k))
    m_j = asarray([rand(d) for i in xrange(T)])
    eps_j = asarray([rand(d) for i in xrange(T)])
    d = rand(1)
    B_0 = d/10 * A_0
    a_j = np.full((T,2,2), fill_value=A_0)
    b_j = np.full((T,2,2), fill_value=B_0)
    phi_i = np.zeros((T, k))
    t1_j, t2_j, t3_j, t4_j = np.ones(k), np.ones((k, 2, 2)), np.ones(k), np.ones(k)
    for t in xrange(T):
        
        for j in range(k):
            t1 = np.sum(psi((1 - k + a_j[t-1])/2)-np.log(abs(b_j[t-1])))
            t1_j[j] = t1            
            for i in xrange(n):
                x_m = (x_train.values[i]-m_j[t-1])
                a_B_j = a_j[t-1] * np.linalg.inv(b_j[t-1])
                t2 = x_m.T* a_B_j * x_m
                t2_j[j] = t2
            t3 = np.trace(a_j[t-1] * np.linalg.inv(b_j[t-1]) * eps_j[t-1])
            t3_j[j] = t3
            t4_j[j] = psi(alpha_k[t-1][j]) - psi(np.sum(alpha_k[t-1]))
            phi_i_num = np.exp(0.5*t1_j[j] - 0.5 * t2_j[j] - 0.5 * t3_j[j] + t4_j[j])
            for _j in range(k):
                phi_i_denum = np.exp(0.5*t1_j[_j] - 0.5 * t2_j[_j] - 0.5 * t3_j[_j] + t4_j[_j])
        print(phi_i_num/phi_i_denum)

In [114]:
if __name__ == '__main__':
    os.chdir('/Users/arkilic/Desktop/')
    xtrn =pd.read_csv('data.txt', header=None)
    vi_gmm(xtrn,8, 10)

[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
[[ 1.  1.]
 [ 1.  1.]]
